In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from skimage import io
from keras import utils
from keras.models import Sequential
from keras.layers import Dense, Flatten,Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras import optimizers

labels= pd.read_csv("train_master.tsv", sep="\t")
master = pd.read_csv("label_master.tsv", sep="\t")
sample = pd.read_csv("sample_submit.csv", header=None, sep=",")

train_images = []
for fname in labels["file_name"]:
    path = "./train/" + fname
    img = io.imread(path)
    train_images.append(img)
train_images = np.array(train_images)
print(type(train_images), train_images.shape)

test_images = []
for fname in sample[0]:
    path = "./test/" + fname
    img = io.imread(path)
    test_images.append(img)
test_images = np.array(test_images )
print(type(test_images ), test_images.shape)

train_images = train_images / 255
test_images = test_images / 255

y = labels["label_id"]

y_categorical = utils.to_categorical(y)
y_categorical

X_con_image, X_ver_image = np.split(train_images, [35000])
y_con_label, y_ver_label = np.split(y_categorical, [35000])

model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3), padding="same", input_shape=(32,32,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=32, kernel_size=(3,3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(units=64))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(units=20))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy",
             optimizer=optimizers.SGD(lr=0.0001, momentum=0.9, decay=0.0, nesterov=True),
              metrics=["accuracy"])

batch_size=32
epochs=60

history = model.fit(X_con_image, y_con_label,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_ver_image, y_ver_label))

Using TensorFlow backend.


<class 'numpy.ndarray'> (50000, 32, 32, 3)
<class 'numpy.ndarray'> (10000, 32, 32, 3)
Train on 35000 samples, validate on 15000 samples
Epoch 1/60
35000/35000 [==============================] - 42s 1ms/step - loss: 3.0050 - accuracy: 0.0521 - val_loss: 2.9949 - val_accuracy: 0.0696
Epoch 2/60
35000/35000 [==============================] - 42s 1ms/step - loss: 2.9954 - accuracy: 0.0515 - val_loss: 2.9949 - val_accuracy: 0.0612
Epoch 3/60
35000/35000 [==============================] - 45s 1ms/step - loss: 2.9939 - accuracy: 0.0546 - val_loss: 2.9946 - val_accuracy: 0.0617
Epoch 4/60
35000/35000 [==============================] - 38s 1ms/step - loss: 2.9927 - accuracy: 0.0557 - val_loss: 2.9943 - val_accuracy: 0.0619
Epoch 5/60
35000/35000 [==============================] - 40s 1ms/step - loss: 2.9917 - accuracy: 0.0571 - val_loss: 2.9937 - val_accuracy: 0.0655
Epoch 6/60
35000/35000 [==============================] - 41s 1ms/step - loss: 2.9903 - accuracy: 0.0582 - val_loss: 2.9928 - val

KeyboardInterrupt: 

In [ ]:
def learning_plot(history, epochs):
    fig = plt.figure(figsize=(15,5))
    plt.subplot(1,1,1)
    plt.plot(range(1,epochs+1), history.history['loss'])
    plt.plot(range(1,epochs+1), history.history['val_loss'])
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.xticks(range(1,epochs+1))
    plt.ylabel('loss')
    plt.legend(['Construction', 'Verification'], loc='upper right')
    plt.show()
    
learning_plot(history,epochs)

In [ ]:
print(model.evaluate(X_ver_image ,y_ver_label))

In [ ]:
pred = model.predict(test_images)

pred = pred.argmax(axis=1)
print(pred)

sample[1] = pred
sample.to_csv("submit9.csv", sep=",", index=None, header=None)